<a href="https://colab.research.google.com/github/vincimech010233/MachineLearningBasics/blob/master/wine_quantum.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
!pip install pennylane
!pip install tensorflow
!pip install transformers  # Para procesamiento de texto
!pip install datasets

In [14]:
import pennylane as qml
from pennylane import numpy as np
import tensorflow as tf
from transformers import BertTokenizer, TFBertModel
from datasets import load_dataset

# Crear un dispositivo cuántico para simulación
dev = qml.device("default.qubit", wires=4)

In [ ]:
dataset = load_dataset('emotion')

train_dataset = dataset['train'].shuffle(seed=42).select(range(100))  # Ejemplo con 100 elementos

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model_bert = TFBertModel.from_pretrained('bert-base-uncased')

def encode_texts(texts):
    return tokenizer(texts, padding=True, truncation=True, return_tensors="tf")

# Imaginemos que tenemos una función que reduce los embeddings de BERT a un tamaño manejable para el circuito cuántico
def reduce_embeddings(embeddings, output_size=4):
    # Esta función debería reducir la dimensionalidad de los embeddings
    # Para el ejemplo, asumiremos que simplemente selecciona algunos elementos
    return embeddings[:, :output_size]

In [17]:
@qml.qnode(dev)
def circuit(features, weights):
    qml.AngleEmbedding(features, wires=range(4))
    qml.StronglyEntanglingLayers(weights, wires=range(4))
    return [qml.expval(qml.PauliZ(i)) for i in range(4)]

In [18]:
class QuantumLayer(tf.keras.layers.Layer):
    def __init__(self):
        super(QuantumLayer, self).__init__()
        self.weights = tf.Variable(
            initial_value=np.random.random(size=(3, 4, 3)), dtype=tf.float32,
            trainable=True
        )

    def call(self, inputs):
        return [circuit(inputs[i], self.weights).numpy() for i in range(inputs.shape[0])]

# Versión simplificada del modelo completo
class HybridModel(tf.keras.Model):
    def __init__(self):
        super(HybridModel, self).__init__()
        self.dense1 = tf.keras.layers.Dense(10, activation="relu")
        self.quantum = QuantumLayer()
        self.dense2 = tf.keras.layers.Dense(1, activation="sigmoid")

    def call(self, inputs):
        x = self.dense1(inputs)
        x = self.quantum(x)
        return self.dense2(x)

In [20]:
class QuantumLayer(tf.keras.layers.Layer):
    def __init__(self):
        super(QuantumLayer, self).__init__()
        self.q_weights = tf.Variable(
            initial_value=np.random.random(size=(3, 4, 3)), dtype=tf.float32,
            trainable=True
        )

    def call(self, inputs):
        # Ajustar la llamada al circuito para usar el nuevo nombre de atributo
        return [circuit(inputs[i], self.q_weights).numpy() for i in range(inputs.shape[0])]


In [21]:
model = HybridModel()

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [22]:
# Supongamos que tenemos una lista de textos y sus etiquetas (esto es hipotético)
texts = ["I feel great", "This is bad"]
labels = [1, 0]  # Suponiendo que 1 es positivo y 0 es negativo

# Tokenización y obtención de embeddings
encoded_texts = encode_texts([t for t in texts])
# "get_bert_embeddings" es una función que te da los embeddings para tus textos
embeddings = model_bert(encoded_texts['input_ids']).last_hidden_state.numpy()

# Reducción de embeddings para que sean manejables por el circuito cuántico
reduced_embeddings = np.array([reduce_embeddings(e) for e in embeddings])

# Convertir las etiquetas a un formato que TensorFlow pueda procesar
labels = np.array(labels)

In [23]:
model.fit(reduced_embeddings, labels, epochs=10, batch_size=2)


Epoch 1/10


ValueError: in user code:

    File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py", line 1401, in train_function  *
        return step_function(self, iterator)
    File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py", line 1384, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py", line 1373, in run_step  **
        outputs = model.train_step(data)
    File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py", line 1150, in train_step
        y_pred = self(x, training=True)
    File "/usr/local/lib/python3.10/dist-packages/keras/src/utils/traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "/tmp/__autograph_generated_filema3e3bvm.py", line 11, in tf__call
        x = ag__.converted_call(ag__.ld(self).quantum, (ag__.ld(x),), None, fscope)
    File

In [24]:
def reduce_embeddings(embeddings, output_size=4):
    reduced_embeddings = embeddings[:, :output_size]
    return reduced_embeddings


In [25]:
reduced_embeddings = reduce_embeddings(embeddings, output_size=4)

# Ahora, `reduced_embeddings` debe tener la forma adecuada para el circuito cuántico

labels = np.array(labels)

# Finalmente, entrena el modelo
model.fit(reduced_embeddings, labels, epochs=10, batch_size=2)

Epoch 1/10
1/1 [==============================] - ETA: 0s

ValueError: Unexpected result of `train_function` (Empty logs). This could be due to issues in input pipeline that resulted in an empty dataset. Otherwise, please use `Model.compile(..., run_eagerly=True)`, or `tf.config.run_functions_eagerly(True)` for more information of where went wrong, or file a issue/bug to `tf.keras`.